In [1]:
from unsloth import FastLanguageModel
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import csv

/vol/bitbucket/kza23/msc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
DIALECT_NAME = "db_dialect"

csv.register_dialect(
    DIALECT_NAME,
    delimiter=",",
    quoting=csv.QUOTE_MINIMAL,
    escapechar="\\",
)

In [3]:
max_seq_length = 20  # Choose any! We auto support RoPE Scaling internally!
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A30 MIG 2g.12gb. Max memory: 11.688 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
codeblocks = pd.read_csv("/vol/bitbucket/kza23/cleaned-up-code.csv")

In [7]:
dataset_verified = load_dataset(
    "msc-smart-contract-audition/vulnerable-functions-base",
    split="train",
    name="verified-functions",
    escapechar="\\",
)

verified_df = dataset_verified.to_pandas()

In [9]:
print(verified_df.head())

                                            function
0  ```\nconstructor(string memory name_, string m...
1  ```\nconstructor(address[] memory tokensToList...
2          ```\nreceive() external payable {}\n```\n
3  ```\nfunction swap(\n  address inputToken,\n  ...
4  ```\nfunction addLiquidity(address inputToken,...


In [14]:
template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below are provided 1 or more code blocks from a smart contract. The task is to explain what is its purpose in plain English.

1. Do NOT use any names of variables, names of parameters or names of functions. Just try to explain the general functionality.
2. Explain what each codeblock does step by step.
3. Give a high-level overview and purpose of the code.
4. First explain the functionality of each code block and after all codeblocks are explained provide a high-level overview of the code over all.

<|start_header_id|>user<|end_header_id|>
Code block to explain:
{}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
Code functionality:
Code block 1:
1. """

template_verified = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below is a single codeblock from a smart contract. The task is to explain what is its purpose in plain English.

1. Do NOT use any names of variables, names of parameters or names of functions. Just try to explain the general functionality.
2. Explain the function does step by step.
3. Give a high-level overview and purpose of the function within a wider context.
4. First explain the functionality of the code block and after provide a high-level overview of the code over all.

<|start_header_id|>user<|end_header_id|>
Code block to explain:
{}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
Code functionality:
Code block 1:
1. """


In [37]:
chunk_len = 100
current_chunk = 0
start = current_chunk * chunk_len
end = (current_chunk + 1) * chunk_len

with open(f"functionality-{current_chunk}.csv", "w", newline="") as f:
    writer = csv.writer(f, dialect=DIALECT_NAME)

    # Write the header row
    writer.writerow(["functionality"])

    for codeblock in tqdm(codeblocks['code'][start:end], total=chunk_len):

        print(codeblock.replace("\\n", "\n"))

        prompt = template.format(codeblock.replace("\\n", "\n"))

        input = tokenizer(prompt, return_tensors="pt", truncation=True).to("cuda")

        output_tokens = model.generate(
            **input, max_new_tokens=512, pad_token_id=tokenizer.pad_token_id
        )

        decoded_output_purpose = tokenizer.decode(
            output_tokens[0],
            skip_special_tokens=True,
            pad_token_id=tokenizer.pad_token_id,
        )

        functionality = decoded_output_purpose.split("Code functionality:\n")[1].strip()
        writer.writerow([functionality.replace("\n", "\\n")])


  0%|                                                                      | 0/100 [00:00<?, ?it/s]

```
@external
@nonreentrant('lock')
def remove_liquidity_one_coin(
    token_amount: uint256, 
    i: uint256, 
    min_amount: uint256, 
    use_eth: bool = False, 
    receiver: address = msg.sender
) -> uint256:
    A_gamma: uint256[2] = self._A_gamma()

    dy: uint256 = 0
    D: uint256 = 0
    p: uint256 = 0
    xp: uint256[N_COINS] = empty(uint256[N_COINS])
    future_A_gamma_time: uint256 = self.future_A_gamma_time
    dy, p, D, xp = self._calc_withdraw_one_coin(A_gamma, token_amount, i, future_A_gamma_time > 0, True)
    assert dy >= min_amount, "Slippage"

    if block.timestamp >= future_A_gamma_time:
        self.future_A_gamma_time = 1

    self.balances[i] -= dy
    CurveToken(self.token).burnFrom(msg.sender, token_amount)

    coin: address = self.coins[i]
    if use_eth and coin == WETH20:
        raw_call(receiver, b"", value=dy)
    else:
        if coin == WETH20:
            WETH(WETH20).deposit(value=dy)
        response: Bytes[32] = raw_call(
            coin, 
  

  0%|                                                                      | 0/100 [00:19<?, ?it/s]

Purpose: Code block 1:
1.  This function is used to remove liquidity from a pool.
2.  It takes in several parameters: the amount of tokens to remove, the index of the coin to remove, the minimum amount of tokens to remove, and an optional boolean flag to specify whether to use ETH or not.
3.  The function calculates the amount of tokens to be removed (`dy`) and the amount of tokens to be transferred (`xp`) based on the pool's parameters and the amount of tokens to remove.
4.  It then checks if the block timestamp is greater than or equal to the future A-gamma time. If it is, it updates the future A-gamma time.
5.  The function then subtracts the amount of tokens to be removed from the pool's balance and burns the tokens from the sender's account.
6.  If the use_eth flag is set to True, it uses the raw_call function to transfer the tokens to the receiver. Otherwise, it uses the raw_call function to transfer the tokens to the receiver, but only if the coin is WETH20. If the coin is not W

In [15]:
chunk_len = 100
current_chunk = 0
start = current_chunk * chunk_len
end = (current_chunk + 1) * chunk_len

with open(f"functionality-verified-{current_chunk}.csv", "w", newline="") as f:
    writer = csv.writer(f, dialect=DIALECT_NAME)

    # Write the header row
    writer.writerow(["functionality"])

    for codeblock in tqdm(verified_df['function'][start:end], total=chunk_len):

        print(codeblock.replace("\\n", "\n"))
        prompt = template_verified.format(codeblock.replace("\\n", "\n"))

        input = tokenizer(prompt, return_tensors="pt", truncation=True).to("cuda")

        output_tokens = model.generate(
            **input, max_new_tokens=512, pad_token_id=tokenizer.pad_token_id
        )

        decoded_output_purpose = tokenizer.decode(
            output_tokens[0],
            skip_special_tokens=True,
            pad_token_id=tokenizer.pad_token_id,
        )

        functionality = decoded_output_purpose.split("Code functionality:\n")[1].strip()
        writer.writerow([functionality.replace("\n", "\\n")])



  0%|                                                                        | 0/100 [00:00<?, ?it/s]

```
constructor(address[] memory tokensToList, uint256 mintAmount, string memory name_, string memory symbol_) ERC20(name_, symbol_) {
  // Basic exchange configuration
  Config memory config;
  config.unlocked = false;
  config.oneMinusTradingFee = 0xffbe76c8b4395800;   // approximately 0.999
  config.delistingBonus = 0;
  DFPconfig = config;

  // Configure the listed tokens as such
  TokenSettings memory listed;
  listed.state = State.Listed;
  require(tokensToList.length == 15, "Incorrect number of tokens");
  address previous = address(0);
  address current = address(0);
  for (uint256 i = 0; i < 15; i++) {
    current = tokensToList[i];
    require(current > previous, "Require ordered list");
    listedTokens[current] = listed;
    previous = current;
  }

  // Generate the LP tokens reflecting the initial liquidity (to be loaded externally)
  _mint(msg.sender, mintAmount);
}
```



  1%|▋                                                               | 1/100 [00:08<14:36,  8.85s/it]

Code block 1:
1.  The code is a constructor function for an ERC20 token.
2.  It initializes the token's configuration and sets the state of the listed tokens.
3.  The configuration includes the initial state of the token (unlocked or locked), the trading fee, and the delisting bonus.
4.  The code checks if the number of tokens provided is correct (15 in this case) and ensures that the list of tokens is in the correct order.
5.  It then sets the state of each token in the list to "Listed" and stores it in a mapping.

High-level overview:
The code is part of a smart contract that manages a token exchange. The constructor function is called when the contract is deployed. It sets the initial state of the token and the listed tokens. The listed tokens are the tokens that are available for trading on the exchange. The code ensures that the list of tokens is correct and in the correct order. The constructor function also generates the LP tokens (Liquidity Provider tokens) reflecting the initi

  2%|█▎                                                              | 2/100 [00:14<11:41,  7.16s/it]

...

Code block 1:
1.  The function `safeIncreaseAllowance` is called with three parameters: `token`, `spender`, and `value`.
2.  It retrieves the current allowance of the `spender` for the `token` using the `allowance` function.
3.  It adds the `value` to the current allowance.
4.  It calls the `approve` function of the `token` contract with the `spender` and the new allowance.
5.  The `_callOptionalReturn` function is used to make the call to the `approve` function. This is done to handle the case where the `approve` function returns a value.

High-level overview and purpose:
The purpose of this function is to increase the allowance of a spender for a specific token. This is a common operation in DeFi (Decentralized Finance) applications, where users need to grant permissions to other users or contracts to spend their tokens.

In the context of a smart contract, this function is used to ensure that the allowance is increased safely. The `_callOptionalReturn` function is used to handle the

 22%|█████████████▊                                                 | 22/100 [03:35<12:40,  9.75s/it]

Code block 1:
1.  The function `safeDecreaseAllowance` is called with three parameters: `token`, `spender`, and `value`.
2.  It first retrieves the current allowance for the `spender` from the `token`.
3.  It then checks if the current allowance is greater than or equal to the `value`. If not, it throws an error.
4.  If the check passes, it calculates the new allowance by subtracting the `value` from the current allowance.
5.  Finally, it calls the `approve` function of the `token` with the `spender` and the new allowance, using the `_callOptionalReturn` function.

High-level overview and purpose:
The purpose of this function is to safely decrease the allowance for a `spender` on a specific `token`. This function ensures that the allowance is not decreased below zero and also handles the approval process with the `token`. This is a common pattern in smart contracts, especially in decentralized finance (DeFi) applications, where tokens are used to manage permissions and access control.


 22%|█████████████▊                                                 | 22/100 [03:47<13:27, 10.36s/it]


KeyboardInterrupt: 

In [2]:
import pandas as pd
import csv

DIALECT_NAME = "db_dialect"
csv.register_dialect(
    DIALECT_NAME,
    delimiter=",",
    quoting=csv.QUOTE_MINIMAL,
    escapechar="\\",
)

# Output file name
output_file = 'functionality.csv'
header = True
# Open the output file in write mode to create/overwrite it
with open(output_file, 'w') as outfile:
    # Iterate through each chunk file
    for i in range(20):
        input_file = f'../functionality-{i}.csv'
        print(f"Processing {input_file}")

        # Read the input CSV file
        df = pd.read_csv(input_file, dialect=DIALECT_NAME)

        # For the first file, include the header
        df.to_csv(output_file, mode='a', index=False, header=header)
        header = False


Processing ../functionality-0.csv
Processing ../functionality-1.csv
Processing ../functionality-2.csv
Processing ../functionality-3.csv
Processing ../functionality-4.csv
Processing ../functionality-5.csv
Processing ../functionality-6.csv
Processing ../functionality-7.csv
Processing ../functionality-8.csv
Processing ../functionality-9.csv
Processing ../functionality-10.csv
Processing ../functionality-11.csv
Processing ../functionality-12.csv
Processing ../functionality-13.csv
Processing ../functionality-14.csv
Processing ../functionality-15.csv
Processing ../functionality-16.csv
Processing ../functionality-17.csv
Processing ../functionality-18.csv
Processing ../functionality-19.csv
